In [16]:
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token
import os 
import logging 
from kiteconnect import KiteConnect
import datetime as dt
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
import ta
import statsmodels.api as sm


In [3]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)




In [4]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [5]:
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [6]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1
    
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data
infy_df = fetchOHLC('INFY','5minute',5)
infy_df.to_csv('infy.csv')
infy_df.head()

,open,high,low,close,volume
date,,,,,
2024-06-05 09:15:00+05:30,1400.15,1418.10,1400.15,1409.45,433610
2024-06-05 09:20:00+05:30,1409.00,1412.65,1404.00,1404.90,290268
2024-06-05 09:25:00+05:30,1404.95,1406.00,1401.95,1405.00,1064215
2024-06-05 09:30:00+05:30,1405.00,1410.05,1403.10,1404.90,321496
2024-06-05 09:35:00+05:30,1404.90,1408.95,1403.05,1404.80,150415


In [7]:
def hammer(df):    
    """returns dataframe with hammer candle column"""
    df = df.copy()
    df["hammer"] = (((df["high"] - df["low"])>3*(df["open"] - df["close"])) & \
                   ((df["close"] - df["low"])/(.001 + df["high"] - df["low"]) > 0.6) & \
                   ((df["open"] - df["low"])/(.001 + df["high"] - df["low"]) > 0.6)) & \
                   (abs(df["close"] - df["open"]) > 0.1* (df["high"] - df["low"]))
    return df

In [8]:
hammer_df = hammer(infy_df)
print(hammer_df['hammer'].value_counts())

hammer
False    252
True      10
Name: count, dtype: int64


In [9]:
def shooting_star(df):    
    """returns dataframe with shooting star candle column"""
    df = df.copy()
    df["shooting_star"] = (((df["high"] - df["low"])>3*(df["open"] - df["close"])) & \
                   ((df["high"] - df["close"])/(.001 + df["high"] - df["low"]) > 0.6) & \
                   ((df["high"] - df["open"])/(.001 + df["high"] - df["low"]) > 0.6)) & \
                   (abs(df["close"] - df["open"]) > 0.1* (df["high"] - df["low"]))
    return df

In [11]:
shooting_star_df = shooting_star(hammer_df)
print(shooting_star_df['shooting_star'].value_counts())

shooting_star
False    247
True      15
Name: count, dtype: int64


In [13]:
def maru_bozu(ohlc_df):    
    """returns dataframe with maru bozu candle column"""
    df = ohlc_df.copy()
    avg_candle_size = abs(df["close"] - df["open"]).median()
    df["h-c"] = df["high"]-df["close"]
    df["l-o"] = df["low"]-df["open"]
    df["h-o"] = df["high"]-df["open"]
    df["l-c"] = df["low"]-df["close"]
    df["maru_bozu"] = np.where((df["close"] - df["open"] > 2*avg_candle_size) & \
                               (df[["h-c","l-o"]].max(axis=1) < 0.005*avg_candle_size),"maru_bozu_green",
                               np.where((df["open"] - df["close"] > 2*avg_candle_size) & \
                               (abs(df[["h-o","l-c"]]).max(axis=1) < 0.005*avg_candle_size),"maru_bozu_red",False))
    df.drop(["h-c","l-o","h-o","l-c"],axis=1,inplace=True)
    return df

In [14]:
maru_bozu_df = maru_bozu(shooting_star_df)

open     high     low      close    volume  hammer  shooting_star  maru_bozu
1400.15  1418.10  1400.15  1409.45  433610  False   False          False        1
1502.60  1504.45  1502.60  1503.85  52870   False   False          False        1
1502.70  1503.00  1501.00  1501.05  75346   False   False          False        1
         1503.30  1499.30  1499.95  187646  False   False          False        1
1502.80  1504.20  1501.45  1502.75  112400  False   False          False        1
                                                                               ..
1449.25  1449.50  1444.00  1445.00  83798   False   False          False        1
1449.50  1450.45  1448.35  1450.00  203297  False   False          False        1
1449.80  1450.85  1449.00  1450.40  79588   False   False          False        1
1450.05  1450.05  1447.50  1447.80  128091  False   False          False        1
1538.25  1538.70  1535.50  1536.00  266295  False   False          False        1
Name: count, Length: 

In [15]:
print(maru_bozu_df['maru_bozu'].value_counts())

maru_bozu
False              259
maru_bozu_green      3
Name: count, dtype: int64


In [18]:
def slope(ohlc_df,n):
    "function to calculate the slope of regression line for n consecutive points on a plot"
    df = ohlc_df.iloc[-1*n:,:]
    y = ((df["open"] + df["close"])/2).values
    x = np.array(range(n))
    y_scaled = (y - y.min())/(y.max() - y.min())
    x_scaled = (x - x.min())/(x.max() - x.min())
    x_scaled = sm.add_constant(x_scaled)
    model = sm.OLS(y_scaled,x_scaled)
    results = model.fit()
    slope = np.rad2deg(np.arctan(results.params[-1]))
    return slope

def trend(ohlc_df,n):
    "function to assess the trend by analyzing each candle"
    df = ohlc_df.copy()
    df["up"] = np.where(df["low"]>=df["low"].shift(1),1,0)
    df["dn"] = np.where(df["high"]<=df["high"].shift(1),1,0)
    if df["close"][-1] > df["open"][-1]:
        if df["up"][-1*n:].sum() >= 0.7*n:
            return "uptrend"
    elif df["open"][-1] > df["close"][-1]:
        if df["dn"][-1*n:].sum() >= 0.7*n:
            return "downtrend"
    else:
        return None

In [19]:
slope = slope(maru_bozu_df,7)
trend = trend(maru_bozu_df,7)

C:\Users\rsurs\AppData\Local\Temp\ipykernel_27148\2097876151.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df["close"][-1] > df["open"][-1]:
C:\Users\rsurs\AppData\Local\Temp\ipykernel_27148\2097876151.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif df["open"][-1] > df["close"][-1]:


In [20]:
slope

-42.5175740835324

In [21]:
trend

'downtrend'